<a href="https://colab.research.google.com/github/edyoda/AI-Agent-Development-and-GenAI/blob/main/Day_6_AI_Agent_Development.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
from google.colab import userdata
WEATHER_API_KEY = userdata.get('WEATHER_API_KEY')
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

In [34]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 6.5 MB/s eta 0:00:00


In [7]:
!pip install langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 23.5 MB/s eta 0:00:00


In [31]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 47.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 45.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 3.3 MB/s eta 0:00:00


In [3]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.3/423.3 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.7 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.49
    Uninstalling langchain-core-0.3.49:
      Successfully uninstalled langchain-core-0.3.49
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.7
    Uninstalling langchain-text-splitters-0.3.7:
      Successfully uninstalled langchain-text-splitters-0.3.7
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.22
    Uninstalling langchain-0.3.22:
      Successfully uninstalled langchain-0.3.22


In [10]:
!pip install pyowm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 48.1 MB/s eta 0:00:00


In [8]:
from langchain_community.document_loaders import DirectoryLoader, TextLoader, PyPDFLoader, CSVLoader
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_openai import OpenAI
from langchain_community.utilities import WikipediaAPIWrapper, SQLDatabase
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain, create_sql_query_chain
from langchain.agents import initialize_agent, Tool, AgentType
from langchain.tools import BaseTool
from langchain.agents.output_parsers import JSONAgentOutputParser

In [23]:
import os
import requests
from typing import Dict, Any
from dotenv import load_dotenv
from langchain.tools import BaseTool

# Load environment variables
# load_dotenv()
# WEATHER_API_KEY = os.getenv("WEATHER_API_KEY")

class WeatherTool(BaseTool):
    name: str = "get_weather"
    description: str = "Get current weather and recommendations for a location"

    def _run(self, location: str) -> str:
        """Get weather information as formatted string"""
        if not WEATHER_API_KEY:
            return "Weather API key not configured"

        try:
            response = requests.get(
                "https://api.weatherapi.com/v1/current.json",
                params={"key": WEATHER_API_KEY, "q": location, "aqi": "no"},
                timeout=10
            )
            response.raise_for_status()
            data = response.json()

            current = data["current"]
            loc = data["location"]
            condition = current["condition"]["text"].lower()

            # Generate recommendations
            umbrella = "rain" in condition or current.get("precip_mm", 0) > 0.5
            jacket = current["temp_c"] < 15
            windy = current["wind_kph"] > 20

            return (
                f"Weather in {loc['name']}, {loc['country']}:\n"
                f"- Temperature: {current['temp_c']}°C ({current['temp_f']}°F)\n"
                f"- Condition: {current['condition']['text']}\n"
                f"- Wind: {current['wind_kph']} km/h\n"
                f"- Humidity: {current['humidity']}%\n\n"
                "Recommendations:\n"
                f"{'- ☔ Take an umbrella (rain expected)' if umbrella else '- No umbrella needed'}\n"
                f"{'- 🧥 Wear a jacket' if jacket else ''}\n"
                f"{'- 🪁 Windy conditions' if windy else ''}\n"
                f"\nLast updated: {current['last_updated']}"
            )
        except Exception as e:
            return f"Error getting weather: {str(e)}"

In [24]:
weather = WeatherTool()

In [26]:
weather._run("Shall I carry umbrella in Bangalore tomorrow")

'Weather in Bangalore, India:\n- Temperature: 31.1°C (88.0°F)\n- Condition: Partly cloudy\n- Wind: 10.4 km/h\n- Humidity: 41%\n\nRecommendations:\n- No umbrella needed\n\n\n\nLast updated: 2025-04-05 18:00'

In [32]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

# Load and chunk documents
loader = TextLoader("/content/data.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

# Create vector store
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)
db = Chroma.from_documents(texts, embeddings)

# Query
query = "What is LangChain?"
docs = db.similarity_search(query)
print(docs[0].page_content)

LangChain was launched in October 2022 as an open source project by Harrison Chase, while working at machine learning startup Robust Intelligence. The project quickly garnered popularity,[3] with improvements from hundreds of contributors on GitHub, trending discussions on Twitter, lively activity on the project's Discord server, many YouTube tutorials, and meetups in San Francisco and London. In April 2023, LangChain had incorporated and the new startup raised over $20 million in funding at a valuation of at least $200 million from venture firm Sequoia Capital, a week after announcing a $10 million seed investment from Benchmark.[4][5]

In the third quarter of 2023, the LangChain Expression Language (LCEL) was introduced, which provides a declarative way to define chains of actions.[6][7]

In October 2023 LangChain introduced LangServe, a deployment tool to host LCEL code as a production-ready API.


In [35]:
from langchain.document_loaders import PyPDFLoader  # Changed from TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter  # Better for PDFs
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

# Load and chunk PDF
loader = PyPDFLoader("/content/Doc.pdf")  # Replace with your PDF path
documents = loader.load()

# Use RecursiveCharacterTextSplitter for better PDF handling
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,  # Overlap helps maintain context
    separators=["\n\n", "\n", " ", ""]  # Split by paragraphs, then sentences
)
texts = text_splitter.split_documents(documents)

# Create vector store
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)
db = Chroma.from_documents(texts, embeddings)

# Query
query = "Give me summary?"
docs = db.similarity_search(query)
print(docs[0].page_content)

We mainly assume that the input is provided in natural
language format, and it contains a high-level description
of the network along with new policies & requirements.
However, by using a multi-modal LLM, it would be possible
to include other information ( e.g., notifications/emails from
a network monitoring service) in other formats ( e.g., the
image/graph of the network topology). Furthermore, since
there is currently no technique to evaluate the correctness
of the translation from the natural language input, the
verifier component can only check the format and syntax
of the generated output or conflicts/bad-practices in the
requirements; the correctness of the output should be
checked by the network operator (see §3 for more details).
2 Generating high-level configurations. Before generat-
ing low-level network configurations, NETBUDDY translates
the formal specification into a high-level configuration.1 By
introducing this intermediate state, N ETBUDDY simplifies


In [37]:
docs[1].page_content

'Making Network Configuration Human Friendly\nChangjie Wang†, Mariano Scazzariello‡, Alireza Farshin∗, Dejan Kosti´c‡, Marco Chiesa‡\n† École Polytechnique‡ KTH Royal Institute of Technology∗ RISE Research Institutes of Sweden\nABSTRACT\nThis paper explores opportunities to utilize Large Language\nModels (LLMs) to make network configuration human-\nfriendly, simplifying the configuration of network devices\nand minimizing errors. We examine the effectiveness of\nthese models in translating high-level policies and require-\nments ( i.e., specified in natural language) into low-level\nnetwork APIs, which requires understanding the hardware\nand protocols. More specifically, we propose N ETBUDDY\nfor generating network configurations from scratch and\nmodifying them at runtime. N ETBUDDY splits the gener-\nation of network configurations into fine-grained steps and\nrelies on self-healing code-generation approaches to better\ntake advantage of the full potential of LLMs. We first'